In [2]:
%%capture
pip install psycopg2 openai ipython-sql python-dotenv

In [3]:
from dotenv import load_dotenv
import os

# Загрузка переменных окружения
load_dotenv()

# Получение значений из .env файла
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")

# Формирование строки подключения
connection_string = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

print("Строка подключения успешно создана (без пароля в выводе).")


Строка подключения успешно создана (без пароля в выводе).


In [6]:
import openai
import psycopg2

# Подключение к PostgreSQL
conn = psycopg2.connect(connection_string)
cursor = conn.cursor()

# # Пример запроса к базе данных
# cursor.execute("SELECT * FROM users;")
# users = cursor.fetchall()
# for user in users:
#     print(user)

# Закрытие соединения
cursor.close()
conn.close()


In [ ]:
# Создание таблицы users
import psycopg2
conn = psycopg2.connect(connection_string)
cursor = conn.cursor()
create_table_query = """
CREATE TABLE IF NOT EXISTS users (
    id SERIAL PRIMARY KEY,
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    patronymic VARCHAR(50),
    birth_year INT CHECK (birth_year BETWEEN 1900 AND 2100),
    city VARCHAR(100),
    role VARCHAR(10) CHECK (role IN ('student', 'teacher')) NOT NULL
);
"""

# Выполнение SQL-запроса
try:
    cursor.execute(create_table_query)
    conn.commit()
    print("Таблица 'users' успешно создана.")
except Exception as e:
    print(f"Ошибка выполнения запроса: {e}")

# # Проверка структуры таблицы
# try:
#     cursor.execute("SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'users';")
#     columns = cursor.fetchall()
#     print("Структура таблицы 'users':")
#     for column in columns:
#         print(column)
# except Exception as e:
#     print(f"Ошибка выполнения запроса: {e}")
    
cursor.close()
conn.close()

Таблица 'users' успешно создана.
Структура таблицы 'users':
('id', 'integer')
('birth_year', 'integer')
('last_name', 'character varying')
('city', 'character varying')
('role', 'character varying')
('patronymic', 'character varying')
('first_name', 'character varying')

In [ ]:
import psycopg2
conn = psycopg2.connect(connection_string)
cursor = conn.cursor()
create_tables_query = """
CREATE TABLE disciplines (
    id SERIAL PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    description TEXT
);

-- Таблица расписания занятий
CREATE TABLE sessions (
    id SERIAL PRIMARY KEY,
    discipline_id INT REFERENCES disciplines(id),
    type VARCHAR(20) CHECK (type IN ('lecture', 'seminar', 'exam')) NOT NULL,
    speaker_id INT REFERENCES users(id),
    start_time TIMESTAMP NOT NULL,
    end_time TIMESTAMP NOT NULL,
    system VARCHAR(50),
    room_id BIGINT
);

-- Таблица связей студентов с занятиями
CREATE TABLE students_sessions (
    id SERIAL PRIMARY KEY,
    student_id INT REFERENCES users(id),
    session_id INT REFERENCES sessions(id)
);

-- Таблица видеозаписей
CREATE TABLE videos (
    id SERIAL PRIMARY KEY,
    session_id INT REFERENCES sessions(id),
    file_path VARCHAR(255) NOT NULL,
    upload_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Таблица аннотаций видеозаписей
CREATE TABLE video_annotations (
    id SERIAL PRIMARY KEY,
    video_id INT REFERENCES videos(id),
    timestamp INTERVAL NOT NULL,
    content TEXT
);

-- Таблица раздаточных материалов
CREATE TABLE materials (
    id SERIAL PRIMARY KEY,
    session_id INT REFERENCES sessions(id),
    file_path VARCHAR(255) NOT NULL,
    upload_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Таблица аннотаций материалов
CREATE TABLE material_annotations (
    id SERIAL PRIMARY KEY,
    material_id INT REFERENCES materials(id),
    page_number INT NOT NULL,
    content TEXT
);
"""
# Выполнение SQL-запроса
try:
    cursor.execute(create_tables_query)
    conn.commit()
    print("Таблицы успешно созданы.")
except Exception as e:
    print(f"Ошибка выполнения запроса: {e}")

cursor.close()
conn.close()

In [8]:

import psycopg2
conn = psycopg2.connect(connection_string)
cursor = conn.cursor()
# Добавление данных в таблицы users, disciplines, sessions
insert_query = """
-- Наполнение таблицы users
INSERT INTO users (first_name, last_name, patronymic, birth_year, city, role) VALUES
('Иван', 'Иванов', 'Иванович', 1998, 'Москва', 'student'),
('Петр', 'Петров', 'Петрович', 1985, 'Саранск', 'student'),
('Игорь', 'Павлов', 'Алексеевич', 1975, 'Москва', 'teacher'),
('Семен', 'Аристархов', 'Павлович', 1980, 'Санкт-Петербург', 'teacher'),
('Иван', 'Иванов', 'Иванович', 1970, 'Новосибирск', 'teacher');

-- Наполнение таблицы disciplines
INSERT INTO disciplines (name, description) VALUES
('Машинное обучение', 'Курс по основам машинного обучения'),
('Проектирование ИС', 'Курс по проектированию информационных систем'),
('Системы ERP', 'Курс по основам ERP-систем');

-- Наполнение таблицы sessions
INSERT INTO sessions (discipline_id, type, speaker_id, start_time, end_time, system, room_id) VALUES
(1, 'lecture', 3, '2024-04-11 14:00', '2024-04-11 17:00', 'zoom', 230340930),
(2, 'lecture', 4, '2024-04-12 19:00', '2024-04-12 20:40', 'zoom', 236740930),
(3, 'exam', 5, '2024-04-13 18:00', '2024-04-13 21:00', 'zoom', 230340930);
"""
# Выполнение SQL-запроса
try:
    cursor.execute(insert_query)
    conn.commit()
    print("Данные успешно добавлены.")
except Exception as e:
    print(f"Ошибка выполнения запроса: {e}")


Данные успешно добавлены.


In [9]:
%%time
cursor.execute("SELECT * FROM users;")
rows = cursor.fetchall()
display(rows)


[(1, 'Иван', 'Иванов', 'Иванович', 1998, 'Москва', 'student'),
 (2, 'Петр', 'Петров', 'Петрович', 1985, 'Саранск', 'student'),
 (3, 'Игорь', 'Павлов', 'Алексеевич', 1975, 'Москва', 'teacher'),
 (4, 'Семен', 'Аристархов', 'Павлович', 1980, 'Санкт-Петербург', 'teacher'),
 (5, 'Иван', 'Иванов', 'Иванович', 1970, 'Новосибирск', 'teacher')]

CPU times: user 4.46 ms, sys: 1.63 ms, total: 6.09 ms
Wall time: 5.58 ms


In [17]:

# Добавление данных в таблицы students_sessions, videos, video_annotations, materials,material_annotations
insert_query = """
-- Наполнение таблицы students_sessions
INSERT INTO students_sessions (student_id, session_id) VALUES
(1, 1),
(2, 1),
(1, 2),
(2, 3);

-- Наполнение таблицы videos
INSERT INTO videos (session_id, file_path) VALUES
(1, '/videos/ml_lecture_20240411.mp4'),
(2, '/videos/is_lecture_20240412.mp4'),
(3, '/videos/erp_exam_20240413.mp4');

-- Наполнение таблицы video_annotations
INSERT INTO video_annotations (video_id, timestamp, content) VALUES
(1, '00:15:00', 'Обсуждение алгоритма регрессии'),
(2, '00:05:00', 'Введение в проектирование ИС'),
(3, '00:30:00', 'Обсуждение основ ERP-систем');

-- Наполнение таблицы materials
INSERT INTO materials (session_id, file_path) VALUES
(1, '/materials/ml_slides.pdf'),
(2, '/materials/is_slides.pdf'),
(3, '/materials/erp_slides.pdf');

-- Наполнение таблицы material_annotations
INSERT INTO material_annotations (material_id, page_number, content) VALUES
(1, 5, 'Пример линейной регрессии'),
(2, 3, 'Диаграмма классов'),
(3, 7, 'Модуль управления финансами');
"""
# Выполнение SQL-запроса
try:
    cursor.execute(insert_query)
    conn.commit()
    print("Данные успешно добавлены.")
except Exception as e:
    print(f"Ошибка выполнения запроса: {e}")


Данные успешно добавлены.


In [4]:

import psycopg2
conn = psycopg2.connect(connection_string)
cursor = conn.cursor()
# Запрос 1: Все лекции, которые начинаются через 5 минут
cursor.execute("SELECT * FROM sessions WHERE type = 'lecture' AND start_time BETWEEN '2024-04-12 18:55' AND NOW() + INTERVAL '5 minutes';")
#("SELECT * FROM sessions WHERE type = 'lecture' AND start_time BETWEEN NOW() AND NOW() + INTERVAL '5 minutes';")
rows = cursor.fetchall()
display(rows)
conn


[(2,
  2,
  'lecture',
  4,
  datetime.datetime(2024, 4, 12, 19, 0),
  datetime.datetime(2024, 4, 12, 20, 40),
  'zoom',
  236740930)]

<connection object at 0x1116478b0; dsn: 'user=Pavel password=xxx dbname=LCMS_DB host=localhost port=5432', closed: 0>

In [5]:
# Запрос 2: Расписание 1 курса на следующую неделю
#import psycopg2
conn = psycopg2.connect(connection_string)
cursor = conn.cursor()
cursor.execute("SELECT s.* FROM sessions s \
              JOIN students_sessions ss ON s.id = ss.session_id \
              JOIN users u ON ss.student_id = u.id \
              WHERE u.role = 'student' AND u.id = 1 \
              AND start_time BETWEEN NOW() - INTERVAL '1 year' AND NOW() + INTERVAL '2 weeks';")
rows = cursor.fetchall()
display(rows)
cursor.close()
conn.commit()
conn.close()

[(1,
  1,
  'lecture',
  3,
  datetime.datetime(2024, 4, 11, 14, 0),
  datetime.datetime(2024, 4, 11, 17, 0),
  'zoom',
  230340930),
 (2,
  2,
  'lecture',
  4,
  datetime.datetime(2024, 4, 12, 19, 0),
  datetime.datetime(2024, 4, 12, 20, 40),
  'zoom',
  236740930)]